In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)
df


In [ ]:
def topsis(df, weights, impacts):
    data = df.iloc[:, 1:].values.astype(float)
    weights = np.array(weights)
    impacts = impacts.split(',')

    norm = data / np.sqrt((data ** 2).sum(axis=0))
    weights = weights / weights.sum()
    weighted = norm * weights

    ideal_best, ideal_worst = [], []
    for i in range(len(impacts)):
        if impacts[i] == '+':
            ideal_best.append(weighted[:, i].max())
            ideal_worst.append(weighted[:, i].min())
        else:
            ideal_best.append(weighted[:, i].min())
            ideal_worst.append(weighted[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)

    d_pos = np.sqrt(((weighted - ideal_best) ** 2).sum(axis=1))
    d_neg = np.sqrt(((weighted - ideal_worst) ** 2).sum(axis=1))

    score = d_neg / (d_pos + d_neg)

    df['Topsis Score'] = score
    df['Rank'] = df['Topsis Score'].rank(ascending=False).astype(int)
    return df


In [ ]:
weights = [1,2,3,4,5]
impacts = "+,+,+,-,+"

result = topsis(df, weights, impacts)
result


In [ ]:
plt.figure(figsize=(8,5))
plt.bar(result.iloc[:,0], result['Topsis Score'])
plt.xticks(rotation=45)
plt.title("TOPSIS Score Comparison")
plt.xlabel("Alternatives")
plt.ylabel("Score")
plt.show()


In [ ]:
result.to_csv("topsis_result.csv", index=False)
files.download("topsis_result.csv")
